# Generating Data

In PyAmpTools, `gen_amp` is a class that uses `AmpTools` to generate events based on some set of amplitudes. `gen_amp` is originally a compiled program, [source](https://github.com/JeffersonLab/halld_sim/blob/master/src/programs/Simulation/gen_amp/gen_amp.cc). AmpTools uses a configuration file to specify the model and dataset to be fitted.


In order to use `gen_amp` within PyAmpTools you must first build the library. Simply go to `external/AMPTOOLS_GENERATORS` and run `make`

In [2]:
import ROOT
import os

############## SET ENVIRONMENT VARIABLES ##############
REPO_HOME     = os.environ['REPO_HOME']
from pyamptools import atiSetup
USE_MPI, USE_GPU, RANK_MPI = atiSetup.setup(globals(), use_genamp=True) # RANK_MPI defaults to 0 even if not using MPI

gen_amp = ROOT.gen_amp

TFile = ROOT.TFile

ModuleNotFoundError: No module named 'ROOT'

## Configuration File

A configuration file contains the specifications of the model (and data) and contains all the information needed by `AmpTools` to generate Monte Carlo data and to perform fits. 

In AmpTools, an amplitude belongs in a coherent sum which itself belongs in to reaction. A unique name for an amplitude follows the format ~ `reactionName::sumName::ampName` where double colons are used to separate the terms. 

If the construction of the amplitudes is complicated with many repeating formats, then a configuration file generator could be used. See [cfgGenerator](https://lan13005.github.io/PyAmpTools/api/cfgGenerator.html) for API and source code.

For now, we can use this generated cfg file

In [2]:
cfgfile = f'{REPO_HOME}/tests/samples/SIMPLE_EXAMPLE/fit.cfg'

In [3]:
%cat $cfgfile

#####################################
####    THIS IS A CONFIG FILE    ####
#####################################
##
##  Blank lines or lines beginning with a "#" are ignored.
##
##  Double colons (::) are treated like a space.
##     This is sometimes useful for grouping (for example,
##     grouping strings like "reaction::sum::amplitudeName")
##
##  All non-comment lines must begin with one of the following keywords.
##
##  (note:  <word> means necessary
##          (word) means optional)
##
##  include       <file>
##  define        <word> (defn1) (defn2) (defn3) ...
##  fit           <fitname>
##  keyword       <keyword> <min arguments> <max arguments>
##  reaction      <reaction> <particle1> <particle2> (particle3) ...
##  data          <reaction> <class> (arg1) (arg2) (arg3) ...
##  genmc         <reaction> <class> (arg1) (arg2) (arg3) ...
##  accmc         <reaction> <class> (arg1) (arg2) (arg3) ...
##  normintfile   <reaction> <file>
##  sum           <reaction> <sum> (sum2) (

Some things to note. The head of the cfg file contains available keywords that provide some specific functionality. See [AmpTools Documentation](https://github.com/mashephe/AmpTools/blob/master/AmpTools_User_Guide.pdf) for more information. The amplitudes used here are known as the Zlm amplitudes. See here for the [Zlm amplitude implementation](https://halldweb.jlab.org/DocDB/0040/004094/003/amptools_polarization.pdf). These complex parameters are initialized in cartesian coordinates. Polar coordinates are are possible

## Generating Events

With the configuration file we can request `gen_amp` to make some simulations.  Not all lines in the above cfg file is needed for generation, for instance the dataset locations, but will be used during fitting.

Here, `gen_amp` is class, and requires instantiation

In [4]:
generator = gen_amp()

`gen_amp` contains many variables to condition the generator. We can see these attribute below

In [5]:
generator.print_help()


Simulation attributes:

----- General -----
configfile:    Configuration file
outname:       Output file name
nEvents:       Number of events to generate
----- Kinematics -----
genFlat:       Generate flat in M(X) (no physics)
lowMass:       Lower edge of mass range (GeV)
highMass:      Upper edge of mass range (GeV)
beamMaxE:      Electron beam energy (or photon energy endpoint)
beamPeakE:     Coherent peak photon energy
beamLowE:      Minimum photon energy to simulate events
beamHighE:     Maximum photon energy to simulate events
slope:         Momentum transfer slope
lowT:          Minimum momentum transfer
highT:         Maximum momentum transfer
----- Output and Conditioning -----
runNum:        Run number assigned to generated events
seed:          Random number seed initialization
diag:          Plot only diagnostic histograms
fsRootFormat:  Enable output in FSRoot format


A good way to set these data members is to create a dictionary to store our specifications (and save if we wish for reproducibility/referencing) and condition our generator as follows. 

These data members requires modification
1. configfile
2. outname

In [6]:
outfile = 'test.root'

conditions = {
    'configfile': cfgfile,
    'outname': outfile,
    'nEvents': 1000,
}

for k, v in conditions.items():
    setattr(generator, k, v)

To see the values of all the datamembers (most contain some default values):

In [7]:
for member in generator.data_members:
    member = str(member)
    print(f'{member:<20} {getattr(generator, member )}' )

centeredVertex       1
diag                 0
genFlat              0
fsRootFormat         0
lowMass              0.2
highMass             2.0
beamMaxE             12.0
beamPeakE            9.0
beamLowE             3.0
beamHighE            12.0
runNum               30731
seed                 0
lowT                 0.0
highT                12.0
slope                6.0
nEvents              15000
batchSize            10000
configfile           /w/halld-scshelf2101/lng/WORK/PyAmpTools/tests/samples/SIMPLE_EXAMPLE/fit.cfg
outname              test.root
hddmname             


After conditioning the generator, simulations can be performed!

In [8]:
generator.generate()

   |        ^                                                      |
   |       / \             Version:  v0.15.1-5-g4c36-dirty         |
   |      /---\                                                    |
   |     /     \           GDouble:  8 bytes                       |
   |    /       \ MP           MPI:  NO                            |
   |     -------               GPU:  NO                            |
   |        |                                                      |
   |        |              doi.org/10.5281/zenodo.5039377          |
   |        | OOLS                                                 |
ConfigFileParser WARNING:  no known resonance found, seed with mass = width = 1GeV
TRandom3 Seed : 4032336773

[ AmplitudeManager ]:

	Creating AmplitudeManager for the reaction:  etapi
		 particle index assignment:  Beam -> 0
		 particle index assignment:  Proton -> 1
		 particle index assignment:  Eta -> 2
		 particle index assignment:  Pi0 -> 3

BeamProperties: Parsing conf

A `*diagnostic.root` is also created containing a variety of mass and angular histograms. A file called `test.root` should have been created containing the 4-momentum components for all particles. To understand physical data collected by a detector, simulations like this one would have to then be passed through a simulation of detector to produce realistic data. We can use `PyROOT` to quickly glance at the contents of the file.

In [9]:
file = ROOT.TFile(outfile)
tree = file.kin

In [10]:
tree.Print()

******************************************************************************
*Tree    :kin       : Kinematics                                             *
*Entries :    15000 : Total =         1269353 bytes  File  Size =     869192 *
*        :          : Tree compression factor =   1.46                       *
******************************************************************************
*Br    0 :NumFinalState : NumFinalState/I                                    *
*Entries :    15000 : Total  Size=      60675 bytes  File Size  =        521 *
*Baskets :        2 : Basket Size=      32000 bytes  Compression= 115.47     *
*............................................................................*
*Br    1 :E_FinalState : E_FinalState[NumFinalState]/F                       *
*Entries :    15000 : Total  Size=     241516 bytes  File Size  =     185633 *
*Baskets :       10 : Basket Size=      32000 bytes  Compression=   1.30     *
*...................................................

Lets clean up these generated files as we do not actually need them

In [11]:
# Free the file handle first so we can clean up
del tree
del file

!rm gen_amp_diagnostic.root
!rm test.root
!rm local_beam.conf